PRF Premium Rates Loader — What this notebook does

TL;DR: This notebook pulls Pasture, Rangeland, Forage (PRF) premium rates from the USDA RMA public API for selected Grid IDs, Years, and Coverage Levels, then writes the results into a Snowflake table. It also adds a couple of helper columns to make sorting/joining by bi-monthly interval + year easier.

Prerequisites

A Snowflake role/warehouse/database/schema you can use.

A mapping table MAP_YTD that links grid IDs (in column SUB_COUNTY_CODE) to state (STATE_CODE) and county (COUNTY_CODE).

The loader looks these up automatically per grid.

Ability to create tables & stored procedures.

Snowflake Python runtime with packages: snowflake-snowpark-python, requests.

What gets created

Target table: CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES
Columns saved per record:

GRID_ID — PRF grid (aka “sub-county” code)

INTENDED_USE — Friendly label (e.g., “Grazing”)

COVERAGE_LEVEL — e.g., 70%, 75%, …

YEAR — e.g., 2025

INDEX_INTERVAL_CODE — one of 625…635 (bi-monthly periods)

INDEX_INTERVAL_NAME — friendly name for the code (e.g., Jan-Feb, Feb-Mar, …, Nov-Dec)

PREMIUMRATE — premium rate returned by the API

INSERT_TIMESTAMP — when the row was written

Python stored procedure:
PUBLIC.LOAD_PRF_PREMIUM_RATES(...) — calls the USDA RMA PRF pricing API and appends rows to the table.

SQL wrapper stored procedure:
PUBLIC.LOAD_RATES_FOR_ALL_PARAMETERS(...) — loops over arrays of Grid IDs, Years, and Coverage Levels, looks up the State/County for each grid, and calls the Python procedure for every combination. Returns a summary of what happened.

Optional helper columns added later:

INTERVAL_MAPPING_TS_TEXT — e.g., Jan-Feb 2025

INTERVAL_MAPPING_TS — numeric key to help sort (e.g., 1.2025 for Jan-Feb 2025)

In [ ]:
USE ROLE ACCOUNTADMIN; -- Or your preferred role
USE WAREHOUSE COMPUTE_WH; -- Or your preferred warehouse
USE DATABASE CAPITAL_MARKETS_SANDBOX;
USE SCHEMA PUBLIC;

In [ ]:
CREATE OR REPLACE TABLE CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES (
    GRID_ID NUMBER,
    INTENDED_USE VARCHAR,
    COVERAGE_LEVEL VARCHAR,
    YEAR NUMBER,
    INDEX_INTERVAL_CODE VARCHAR, -- This will hold '625', '626', etc.
    INDEX_INTERVAL_NAME VARCHAR, -- This will hold 'Jan-Feb', 'Feb-Mar', etc.
    PREMIUMRATE FLOAT,
    INSERT_TIMESTAMP TIMESTAMP_NTZ
);

In [ ]:
CREATE OR REPLACE PROCEDURE CAPITAL_MARKETS_SANDBOX.PUBLIC.LOAD_PRF_PREMIUM_RATES(
    COVERAGE_LEVEL_PERCENT INT,
    GRID_ID_PARAM STRING,
    INTENDED_USE_PARAM STRING,
    YEAR_PARAM INT,
    STATE_CODE_PARAM STRING,
    COUNTY_CODE_PARAM STRING,
    IRRIGATION_PRACTICE_CODE_PARAM STRING,
    ORGANIC_PRACTICE_CODE_PARAM STRING,
    INTENDED_USE_CODE_PARAM STRING,
    INTERVAL_PERCENT_OF_VALUES_PARAM STRING
)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.10'
PACKAGES = ('snowflake-snowpark-python', 'requests')
HANDLER = 'main'
EXTERNAL_ACCESS_INTEGRATIONS = (usda_api_integration) 
AS
$$
import snowflake.snowpark as snowpark
import requests
import datetime

def main(session: snowpark.Session, 
         coverage_level_percent: int,
         grid_id_param: str,
         intended_use_param: str,
         year_param: int,
         state_code_param: str,
         county_code_param: str,
         irrigation_practice_code_param: str,
         organic_practice_code_param: str,
         intended_use_code_param: str,
         interval_percent_of_values_param: str
         ) -> str:
    
    # Mapping dictionary to get friendly names
    interval_mapping = {
        "625": "Jan-Feb", "626": "Feb-Mar", "627": "Mar-Apr", "628": "Apr-May",
        "629": "May-Jun", "630": "Jun-Jul", "631": "Jul-Aug", "632": "Aug-Sep",
        "633": "Sep-Oct", "634": "Oct-Nov", "635": "Nov-Dec"
    }

    PARAMS = {
        "intervalType": "BiMonthly",
        "irrigationPracticeCode": irrigation_practice_code_param, 
        "organicPracticeCode": organic_practice_code_param,
        "intendedUseCode": intended_use_code_param, 
        "stateCode": state_code_param,
        "countyCode": county_code_param,
        "productivityFactor": "100",
        "insurableInterest": "100",  
        "insuredAcres": "1000",
        "sampleYear": str(year_param),
        "intervalPercentOfValues": interval_percent_of_values_param,
        "coverageLevelPercent": str(coverage_level_percent),
        "gridId": grid_id_param,
        "gridName": grid_id_param
    }
    
    base_url = "https://public-rma.fpac.usda.gov/apps/PrfWebApi/PrfExternalPricingRates/GetPricingRates"
    target_table = "CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES"
    run_timestamp = datetime.datetime.now()
    
    try:
        response = requests.get(base_url, params=PARAMS, timeout=60)
        response.raise_for_status() 
        api_data = response.json()
        
        if 'returnData' not in api_data or 'PricingRateRows' not in api_data['returnData']:
            return f"Error: 'returnData' or 'PricingRateRows' not found. Response: {api_data}"
        
        table_rows = api_data['returnData']['PricingRateRows']
        
        if not table_rows:
            return f"No data returned for Grid {grid_id_param}, Year {year_param}, Level {coverage_level_percent}%."
            
        processed_data = []
        
        for row in table_rows:
            interval_code = row.get('IntervalCode')
            if interval_code and interval_code != 'Total':
                interval_name = interval_mapping.get(interval_code, interval_code) 
                processed_row = (
                    int(grid_id_param),
                    intended_use_param,
                    f"{coverage_level_percent}%",
                    year_param,
                    interval_code,
                    interval_name,
                    row.get('PremiumRate'),
                    run_timestamp
                )
                processed_data.append(processed_row)
        
        if processed_data:
            column_names = [
                "GRID_ID", "INTENDED_USE", "COVERAGE_LEVEL", "YEAR", 
                "INDEX_INTERVAL_CODE", "INDEX_INTERVAL_NAME", 
                "PREMIUMRATE", "INSERT_TIMESTAMP"
            ]
            df = session.create_dataframe(processed_data, schema=column_names)
            df.write.mode("append").save_as_table(target_table)
            return f"Success: Loaded {len(processed_data)} rows for Grid {grid_id_param}, Year {year_param}, Level {coverage_level_percent}%."
        else:
            return f"Success: No valid premium rate rows found for Grid {grid_id_param}, Year {year_param}, Level {coverage_level_percent}%."
            
    except Exception as e:
        return f"An error occurred for Grid {grid_id_param}, Year {year_param}, Level {coverage_level_percent}%: {str(e)}"
$$;

In [ ]:
CREATE OR REPLACE PROCEDURE CAPITAL_MARKETS_SANDBOX.PUBLIC.LOAD_RATES_FOR_ALL_PARAMETERS(
    GRIDS_ARRAY ARRAY,
    YEARS_ARRAY ARRAY,
    COVERAGE_LEVELS_ARRAY ARRAY,
    INTENDED_USE_PARAM STRING,
    IRRIGATION_PRACTICE_CODE_PARAM STRING,
    ORGANIC_PRACTICE_CODE_PARAM STRING,
    INTENDED_USE_CODE_PARAM STRING,
    INTERVAL_PERCENT_OF_VALUES_PARAM STRING
)
RETURNS VARIANT
LANGUAGE SQL
AS
$$
DECLARE
    GRID_ID STRING;
    YEAR INT;
    COVERAGE_LEVEL INT;
    
    -- Local variables to hold the looked-up values
    V_STATE_CODE STRING;
    V_COUNTY_CODE STRING;
    
    RESULTS ARRAY DEFAULT ARRAY_CONSTRUCT();
    RESULT_MSG VARCHAR;
BEGIN
    FOR I IN 0 TO ARRAY_SIZE(GRIDS_ARRAY) - 1 DO
        GRID_ID := GRIDS_ARRAY[I];
        
        -- === NEW LOOKUP STEP ===
        -- Get the State and County for the current Grid ID
        -- We use DISTINCT in case a grid appears multiple times
        SELECT DISTINCT
            STATE_CODE, COUNTY_CODE
        INTO
            :V_STATE_CODE, :V_COUNTY_CODE
        FROM 
            CAPITAL_MARKETS_SANDBOX.PUBLIC.MAP_YTD
        WHERE 
            SUB_COUNTY_CODE = :GRID_ID
        LIMIT 1;
        -- ========================
        
        FOR J IN 0 TO ARRAY_SIZE(YEARS_ARRAY) - 1 DO
            YEAR := YEARS_ARRAY[J];
            
            FOR K IN 0 TO ARRAY_SIZE(COVERAGE_LEVELS_ARRAY) - 1 DO
                COVERAGE_LEVEL := COVERAGE_LEVELS_ARRAY[K];
                
                -- Pass the looked-up variables to the Python procedure
                CALL CAPITAL_MARKETS_SANDBOX.PUBLIC.LOAD_PRF_PREMIUM_RATES(
                    :COVERAGE_LEVEL,
                    :GRID_ID,
                    :INTENDED_USE_PARAM,
                    :YEAR,
                    :V_STATE_CODE,   -- Pass the variable
                    :V_COUNTY_CODE,  -- Pass the variable
                    :IRRIGATION_PRACTICE_CODE_PARAM,
                    :ORGANIC_PRACTICE_CODE_PARAM,
                    :INTENDED_USE_CODE_PARAM,
                    :INTERVAL_PERCENT_OF_VALUES_PARAM
                ) INTO :RESULT_MSG;
                
                RESULTS := ARRAY_APPEND(RESULTS, OBJECT_CONSTRUCT(
                    'grid_id', GRID_ID, 
                    'year', YEAR, 
                    'coverage_level', COVERAGE_LEVEL, 
                    'status', RESULT_MSG
                ));
                
            END FOR;
        END FOR;
    END FOR;

    RETURN OBJECT_CONSTRUCT(
        'message', 'Completed processing all parameters.',
        'total_calls_made', ARRAY_SIZE(RESULTS),
        'results_per_call', RESULTS
    );
END;
$$;

In [ ]:
-- --- Static Parameters ---
SET INTENDED_USE_TO_LOAD = 'Grazing';
SET IRRIGATION_PRACTICE_CODE_TO_LOAD = '997';
SET ORGANIC_PRACTICE_CODE_TO_LOAD = '997';
SET INTENDED_USE_CODE_TO_LOAD = '007';
SET INTERVAL_PERCENT_OF_VALUES_TO_LOAD = '[50,0,50,0,0,0,0,0,0,0,0]';

In [ ]:
-- --- Execute the Main Wrapper (Full Grid List for 2025) ---
CALL CAPITAL_MARKETS_SANDBOX.PUBLIC.LOAD_RATES_FOR_ALL_PARAMETERS(
    -- --- Your Full List of Grids ---
    ARRAY_CONSTRUCT(
        9128, 9129, 9130, 9131, 
        8828, 8829, 8830, 8831, 
        8528, 8529, 8530, 8531, 
        8228, 8229, 8230, 8231, 
        7928, 7929, 7930, 7931
    ),
    
    -- --- Your List of Years (2025 Only) ---
    ARRAY_CONSTRUCT(2025),
    
    -- --- Your List of Coverage Levels ---
    ARRAY_CONSTRUCT(70, 75, 80, 85, 90),
    
    -- --- Parameters from previous cell ---
    $INTENDED_USE_TO_LOAD,
    $IRRIGATION_PRACTICE_CODE_TO_LOAD,
    $ORGANIC_PRACTICE_CODE_TO_LOAD,
    $INTENDED_USE_CODE_TO_LOAD,
    $INTERVAL_PERCENT_OF_VALUES_TO_LOAD
);

In [ ]:
-- --- Query Your Results ---
SELECT * FROM CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES
ORDER BY GRID_ID, YEAR, COVERAGE_LEVEL, INDEX_INTERVAL_CODE;

In [ ]:
-- Step 1: Add the new columns (if they don't exist)
ALTER TABLE IF EXISTS CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES
ADD COLUMN IF NOT EXISTS INTERVAL_MAPPING_TS_TEXT VARCHAR;

ALTER TABLE IF EXISTS CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES
ADD COLUMN IF NOT EXISTS INTERVAL_MAPPING_TS NUMBER(8,4);


-- Step 2: Populate the new columns
UPDATE CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES
SET
    -- Create the text version, e.g., 'Jan-Feb 1950'
    INTERVAL_MAPPING_TS_TEXT = INDEX_INTERVAL_NAME || ' ' || YEAR,
    
    -- Create the numeric version, e.g., 1.1950
    INTERVAL_MAPPING_TS = (
        -- Get the interval number
        CASE INDEX_INTERVAL_NAME
            WHEN 'Jan-Feb' THEN 1
            WHEN 'Feb-Mar' THEN 2
            WHEN 'Mar-Apr' THEN 3
            WHEN 'Apr-May' THEN 4
            WHEN 'May-Jun' THEN 5
            WHEN 'Jun-Jul' THEN 6
            WHEN 'Jul-Aug' THEN 7
            WHEN 'Aug-Sep' THEN 8
            WHEN 'Sep-Oct' THEN 9
            WHEN 'Oct-Nov' THEN 10
            WHEN 'Nov-Dec' THEN 11
            ELSE NULL -- Handle any other cases
        END
    ) + (YEAR / 10000.0) -- Combine with year (e.g., 1 + (1950 / 10000.0) = 1.1950)
WHERE
    -- Only update rows that haven't been processed yet
    INTERVAL_MAPPING_TS IS NULL 
    OR INTERVAL_MAPPING_TS_TEXT IS NULL;

In [ ]:
SELECT * FROM CAPITAL_MARKETS_SANDBOX.PUBLIC.PRF_PREMIUM_RATES
LIMIT 1200;